In [ ]:
from google.colab import drive
drive.flush_and_unmount()   # unmount previous mount safely
!rm -rf /content/drive      # remove old mount folder
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    ResNet50, VGG16, InceptionV3, EfficientNetB0, MobileNetV2
)
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Paths
BASE_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification"
SAVE_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/model_results"
LOG_FILE = os.path.join(SAVE_DIR, "training_log.csv")
# Make sure save directory exists
os.makedirs(SAVE_DIR, exist_ok=True)
# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
# Data Generators
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(BASE_DIR, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(BASE_DIR, "val"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(BASE_DIR, "test"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

In [ ]:
def build_model(base_model_class, input_shape=(224, 224, 3)):
    base_model = base_model_class(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_classes = {
    "ResNet50": ResNet50,
    "VGG16": VGG16,
    "InceptionV3": InceptionV3,
    "EfficientNetB0": EfficientNetB0,
    "MobileNetV2": MobileNetV2
}
# Prepare log DataFrame
log_records = []
#Training Loop
for model_name, model_class in model_classes.items():
    print(f"\nTraining {model_name}...\n")

    model = build_model(model_class)

    # Callbacks
    checkpoint_path = os.path.join(SAVE_DIR, f"best_{model_name}.h5")
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss"),
        ModelCheckpoint(checkpoint_path, save_best_only=True, monitor="val_loss")
    ]
    # Train model
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        callbacks=callbacks,
        verbose=1
    )
    # Save final model
    final_model_path = os.path.join(SAVE_DIR, f"final_{model_name}.h5")
    model.save(final_model_path)
    # Log metrics
    from datetime import datetime
    for epoch in range(len(history.history["accuracy"])):
        log_records.append({
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "model": model_name,
            "epoch": epoch + 1,
            "accuracy": history.history["accuracy"][epoch],
            "val_accuracy": history.history["val_accuracy"][epoch],
            "loss": history.history["loss"][epoch],
            "val_loss": history.history["val_loss"][epoch],
        })

    print(f"Finished {model_name}. Model saved at {final_model_path}")
#Save Log to CSV
df_log = pd.DataFrame(log_records)
df_log.to_csv(LOG_FILE, index=False)

print(f"\nTraining complete! Logs saved to {LOG_FILE}")

In [ ]:
import os

search_folder = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/non_defective"

# List all files that contain "Electrical"
for f in os.listdir(search_folder):
  print(f)


In [ ]:
from tensorflow.keras.models import load_model
MODEL_PATH = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/model_results/best_MobileNetV2.h5"
model = load_model(MODEL_PATH)
print("Model loaded successfully!")
import numpy as np
from tensorflow.keras.preprocessing import image
IMG_SIZE = (224, 224)
img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/non_defective/Dust (167).jpg"
# Load and preprocess
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # rescale
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)[0][0]
if prediction >= 0.5:
    label = "Non-Defective"
else:
    label = "Defective"
print(f"Prediction: {label} (Confidence: {prediction:.4f})")


In [ ]:
import os

search_folder = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/defective"

# List all files that contain "Electrical"
for f in os.listdir(search_folder):
  print(f)


In [ ]:
from tensorflow.keras.models import load_model
MODEL_PATH = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/model_results/best_MobileNetV2.h5"
model = load_model(MODEL_PATH)
print("Model loaded successfully!")
import numpy as np
from tensorflow.keras.preprocessing import image
IMG_SIZE = (224, 224)
img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/defective/Electrical (30).jpg"
# Load and preprocess
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # rescale
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)[0][0]
if prediction >= 0.5:
    label = "Non-Defective"
else:
    label = "Defective"
print(f"Prediction: {label} (Confidence: {prediction:.4f})")


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    ResNet50, VGG16, InceptionV3, EfficientNetB0, MobileNetV2
)
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
BASE_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses"
SAVE_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results"
LOG_FILE = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results/training_log.csv"
os.makedirs(SAVE_DIR, exist_ok=True)
BATCHES = [f"SplitBalanced_batch_{i}" for i in range(1, 5)]
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
def get_model_list(input_shape=(224, 224, 3)):
    num_classes = 1
    models_list = []

    def build_model(base_model, name):
        x = layers.GlobalAveragePooling2D()(base_model.output)
        x = layers.Dense(128, activation='relu')(x)
        output = layers.Dense(num_classes, activation='sigmoid')(x)
        model = models.Model(inputs=base_model.input, outputs=output, name=name)
        return model

    bases = [
        (ResNet50, "ResNet50"),
        (VGG16, "VGG16"),
        (InceptionV3, "InceptionV3"),
        (EfficientNetB0, "EfficientNetB0"),
        (MobileNetV2, "MobileNetV2"),
    ]

    for cls, name in bases:
        base = cls(weights='imagenet', include_top=False, input_shape=input_shape)
        models_list.append(build_model(base, name))

    return models_list

In [ ]:
results = []
for batch_name in BATCHES:
    print(f"\n===== 🔹 Training on {batch_name} 🔹 =====")
    batch_path = os.path.join(BASE_DIR, batch_name)
    train_gen = train_datagen.flow_from_directory(
        os.path.join(batch_path, "train"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    val_gen = val_test_datagen.flow_from_directory(
        os.path.join(batch_path, "val"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    test_gen = val_test_datagen.flow_from_directory(
        os.path.join(batch_path, "test"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    models_list = get_model_list()
    for model in models_list:
        save_path = f"{SAVE_DIR}/{batch_name}_{model.name}.keras"
        if os.path.exists(save_path):
            print(f"⏭Skipping {model.name} — already trained for {batch_name}")
            continue
        print(f"\nTraining model: {model.name} on {batch_name}")
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
            ModelCheckpoint(save_path, monitor='val_accuracy', save_best_only=True)
        ]
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=EPOCHS,
            callbacks=callbacks,
            verbose=1
        )
        # Evaluate on test set
        test_loss, test_acc = model.evaluate(test_gen, verbose=0)
        print(f"{model.name} test accuracy on {batch_name}: {test_acc:.4f}")

        results.append({
            "Batch": batch_name,
            "Model": model.name,
            "Test Accuracy": round(test_acc, 4)
        })
        pd.DataFrame(results).to_csv(LOG_FILE, index=False)
        print(f"Logged results to {LOG_FILE}")


In [ ]:
pd.read_csv(LOG_FILE)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

def plot_model_curves(model_path, train_gen, val_gen):
    print(f"\n📊 Plotting training curves for {os.path.basename(model_path)}")

    # Reload model
    model = load_model(model_path)

    # Recompile to ensure metrics available
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Retrain just for plotting (few epochs, no saving)
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=3,
        verbose=1
    )

    plt.figure(figsize=(10,4))

    # Accuracy Plot
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train acc', marker='o')
    plt.plot(history.history['val_accuracy'], label='val acc', marker='o')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss Plot
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train loss', marker='o')
    plt.plot(history.history['val_loss'], label='val loss', marker='o')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
import tensorflow as tf
import gc
from tensorflow.keras import backend as K
results = []
for batch_name in BATCHES:
    print(f"\n===== 🔹 Training on {batch_name} 🔹 =====")
    batch_path = os.path.join(BASE_DIR, batch_name)

    train_gen = train_datagen.flow_from_directory(
        os.path.join(batch_path, "train"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    val_gen = val_test_datagen.flow_from_directory(
        os.path.join(batch_path, "val"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    test_gen = val_test_datagen.flow_from_directory(
        os.path.join(batch_path, "test"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    models_list = get_model_list()

    for model in models_list:
        save_path = f"{SAVE_DIR}/{batch_name}_{model.name}.keras"

        if os.path.exists(save_path):
            print(f"⏭Skipping {model.name} — already trained for {batch_name}")
            continue

        print(f"\nTraining model: {model.name} on {batch_name}")
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        )

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            save_path,
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=EPOCHS,
            callbacks=[early_stopping, checkpoint],
            verbose=1
        )

        # Determine last and best epoch
        stopped_epoch = len(history.history['loss'])
        best_val_acc = max(history.history['val_accuracy'])
        best_val_loss = min(history.history['val_loss'])

        print(f"Stopped at epoch: {stopped_epoch}")
        print(f"Best Val Accuracy: {best_val_acc:.4f}, Best Val Loss: {best_val_loss:.4f}")

        # Evaluate on test data
        test_loss, test_acc = model.evaluate(test_gen, verbose=0)
        print(f"Test Accuracy on {batch_name}: {test_acc:.4f}")

        # Log results
        results.append({
            "Batch": batch_name,
            "Model": model.name,
            "Stopped Epoch": stopped_epoch,
            "Best Val Accuracy": round(best_val_acc, 4),
            "Best Val Loss": round(best_val_loss, 4),
            "Test Accuracy": round(test_acc, 4)
        })

        pd.DataFrame(results).to_csv(LOG_FILE, index=False)
        print(f"Logged results to {LOG_FILE}")

        # ✅ Free GPU memory
        print("Clearing GPU memory...")
        del model
        K.clear_session()
        gc.collect()


In [ ]:
# Print last epoch accuracy and validation accuracy
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

# Or visualize the accuracy curve
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()


In [ ]:
!ls "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results"


In [ ]:
import os
import pandas as pd

# Paths
results_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results"
log_path = os.path.join(results_dir, "training_log.csv")

# Load your log
df = pd.read_csv(log_path)
logged_models = set(df["Model"].astype(str) + "_" + df["Batch"].astype(str))

# Find all .keras files
all_models = []
for f in os.listdir(results_dir):
    if f.endswith(".keras"):
        parts = f.replace(".keras", "").split("_")
        batch = "_".join(parts[0:2])  # e.g. SplitBalanced_batch_1
        model = parts[2]              # e.g. ResNet50
        all_models.append((batch, model))

# Find missing
missing = [(b, m) for (b, m) in all_models if f"{m}_{b}" not in logged_models]
print("🔍 Missing entries in training_log.csv:")
for b, m in missing:
    print(f"- {b} → {m}")


In [ ]:
import os
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Paths
results_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results"
base_dir = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses"
log_path = os.path.join(results_dir, "training_log.csv")

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Load existing log (if available)
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
else:
    df = pd.DataFrame(columns=["Batch", "Model", "Test Accuracy", "Best Val Accuracy", "Best Val Loss"])

# Existing entries
logged_pairs = set(zip(df["Batch"], df["Model"]))

# Prepare test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

new_rows = []

# Iterate over all model files
for file in os.listdir(results_dir):
    if not file.endswith(".keras"):
        continue

    # Example file: SplitBalanced_batch_3_ResNet50.keras
    parts = file.replace(".keras", "").split("_")
    batch = "_".join(parts[0:3]) if len(parts) >= 3 else parts[0]
    model_name = parts[-1]

    if (batch, model_name) in logged_pairs:
        print(f"⏭ Skipping {model_name} on {batch} — already logged")
        continue

    model_path = os.path.join(results_dir, file)
    test_dir = os.path.join(base_dir, batch, "test")

    if not os.path.exists(test_dir):
        print(f"Test folder not found for {batch}, skipping...")
        continue

    print(f"Evaluating {model_name} on {batch} ...")

    try:
        model = load_model(model_path)

        test_gen = test_datagen.flow_from_directory(
            test_dir,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='binary',
            shuffle=False
        )

        loss, acc = model.evaluate(test_gen, verbose=0)
        print(f"{model_name} on {batch} — Test Accuracy: {acc:.4f}")

        new_rows.append({
            "Batch": batch,
            "Model": model_name,
            "Test Accuracy": acc,
            "Best Val Accuracy": np.nan,
            "Best Val Loss": np.nan
        })

    except Exception as e:
        print(f"Error evaluating {model_name} on {batch}: {e}")

# Append new results to CSV
if new_rows:
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    df.to_csv(log_path, index=False)
    print(f"\nUpdated training_log.csv with {len(new_rows)} new entries.")
else:
    print("\nAll models already logged.")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(log_path)
plt.figure(figsize=(10,6))
sns.barplot(data=df, x="Model", y="Test Accuracy", hue="Batch")
plt.title("Test Accuracy by Model and Batch")
plt.xticks(rotation=30)
plt.show()


In [ ]:
import pandas as pd

LOG_FILE = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results/training_log.csv"
df = pd.read_csv(LOG_FILE)
print(df)


In [ ]:
BEST_MODEL_PATH = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/model_results/SplitBalanced_batch_4_MobileNetV2.keras"
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model(BEST_MODEL_PATH)
print("Best model loaded successfully!")

IMG_SIZE = (224, 224)

# Try a test image
img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/defective/Electrical (30).jpg"

img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)[0][0]
label = "Non-Defective" if prediction >= 0.5 else "Defective"
print(f"Prediction: {label} (Confidence: {prediction:.4f})")
import matplotlib.pyplot as plt

plt.imshow(img_array[0])
plt.title(f"{label} ({prediction:.2f})")
plt.axis('off')
plt.show()


In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    ResNet50, VGG16, InceptionV3, EfficientNetB0, MobileNetV2
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
BASE_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses"
SAVE_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results"
LOG_FILE = os.path.join(SAVE_DIR, "training_log.csv")
os.makedirs(SAVE_DIR, exist_ok=True)
BATCHES = [f"SplitBalanced_batch_{i}" for i in range(1, 5)]
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.25,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        bce_exp = tf.exp(-bce)
        return alpha * (1 - bce_exp) ** gamma * bce
    return focal_loss_fixed
def get_model_list(input_shape=(224, 224, 3)):
    models_list = []
    def build_model(base_model, name):
        for layer in base_model.layers[:-30]:
            layer.trainable = False
        for layer in base_model.layers[-30:]:
            layer.trainable = True
        x = layers.GlobalAveragePooling2D()(base_model.output)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.4)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        output = layers.Dense(1, activation='sigmoid')(x)
        model = models.Model(inputs=base_model.input, outputs=output, name=name)
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                      loss=focal_loss(), metrics=['accuracy'])
        return model
    bases = [
        (ResNet50, "ResNet50"),
        (VGG16, "VGG16"),
        (InceptionV3, "InceptionV3"),
        (EfficientNetB0, "EfficientNetB0"),
        (MobileNetV2, "MobileNetV2"),
    ]
    for cls, name in bases:
        base = cls(weights='imagenet', include_top=False, input_shape=input_shape)
        models_list.append(build_model(base, name))
    return models_list
results = []
for batch_name in BATCHES:
    print(f"\n===== 🔹 Training on {batch_name} 🔹 =====")
    batch_path = os.path.join(BASE_DIR, batch_name)
    train_gen = train_datagen.flow_from_directory(
        os.path.join(batch_path, "train"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    val_gen = val_test_datagen.flow_from_directory(
        os.path.join(batch_path, "val"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    test_gen = val_test_datagen.flow_from_directory(
        os.path.join(batch_path, "test"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    models_list = get_model_list()
    for model in models_list:
        save_path = f"{SAVE_DIR}/{batch_name}_{model.name}.keras"
        if os.path.exists(save_path):
            print(f"⏭ Skipping {model.name} — already trained for {batch_name}")
            continue
        print(f"\nTraining model: {model.name} on {batch_name}")
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
            ModelCheckpoint(save_path, monitor='val_accuracy', save_best_only=True, verbose=1)
        ]
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=EPOCHS,
            callbacks=callbacks,
            verbose=1,
            class_weight={0: 1.0, 1: 1.8}
        )
        # Evaluate on test data
        test_loss, test_acc = model.evaluate(test_gen, verbose=0)
        best_val_acc = max(history.history['val_accuracy'])
        best_val_loss = min(history.history['val_loss'])
        print(f"{model.name} on {batch_name}: Test Accuracy={test_acc:.4f}, Best Val Acc={best_val_acc:.4f}")
        results.append({
            "Batch": batch_name,
            "Model": model.name,
            "Best Val Accuracy": round(best_val_acc, 4),
            "Best Val Loss": round(best_val_loss, 4),
            "Test Accuracy": round(test_acc, 4)
        })
        pd.DataFrame(results).to_csv(LOG_FILE, index=False)
        print(f"Logged results to {LOG_FILE}")
        del model
        K.clear_session()
        gc.collect()

print("\Training completed successfully!")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# --- define focal loss again ---
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        bce_exp = tf.exp(-bce)
        return alpha * (1 - bce_exp) ** gamma * bce
    return focal_loss_fixed

BEST_MODEL_PATH = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results/SplitBalanced_batch_4_ResNet50.keras"
model = load_model(BEST_MODEL_PATH, custom_objects={'focal_loss_fixed': focal_loss()})
IMG_SIZE = (224, 224)

img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/defective/Physical (42).jpg"
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)[0][0]
label = "Non-Defective" if prediction >= 0.5 else "Defective"

plt.imshow(img_array[0])
plt.title(f"{label} ({prediction:.2f})")
plt.axis('off')
plt.show()


In [ ]:
!ls /content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/SplitBalanced_batch_4/

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# --- define focal loss again ---
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        bce_exp = tf.exp(-bce)
        return alpha * (1 - bce_exp) ** gamma * bce
    return focal_loss_fixed

BEST_MODEL_PATH = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results/SplitBalanced_batch_4_MobileNetV2.keras"
model = load_model(BEST_MODEL_PATH, custom_objects={'focal_loss_fixed': focal_loss()})
IMG_SIZE = (224, 224)

img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/SplitBalanced_batch_1/test/non_defective/Clean (42).jpg"
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)[0][0]
label = "Non-Defective" if prediction >= 0.5 else "Defective"

plt.imshow(img_array[0])
plt.title(f"{label} ({prediction:.2f})")
plt.axis('off')
plt.show()


In [ ]:
import pandas as pd

LOG_FILE = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results/training_log.csv"

# Load the log file
df = pd.read_csv(LOG_FILE)
print("🔹 All Results:")
display(df)

# Sort by Test Accuracy (or Val Accuracy if you prefer)
best = df.sort_values(by="Test Accuracy", ascending=False).head(10)
print("\nTop Performing Models:")
display(best)

# Get the single best one
best_model_row = best.iloc[0]
print("\nBest Overall:")
print(best_model_row)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
for model in df['Model'].unique():
    subset = df[df['Model'] == model]
    plt.plot(subset['Batch'], subset['Test Accuracy'], marker='o', label=model)

plt.title("Model Performance Across Batches")
plt.xlabel("Batch")
plt.ylabel("Test Accuracy")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Average
from tensorflow.keras import Model, Input
import tensorflow as tf
import os # Import the os module

# --- Re-declare focal loss (needed to load models) ---
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        bce_exp = tf.exp(-bce)
        return alpha * (1 - bce_exp) ** gamma * bce
    return focal_loss_fixed

# --- Path to your saved models ---
MODEL_DIR = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results"
batch_name = "SplitBalanced_batch_3"  # (or whichever batch gave best results)

model_paths = [
    f"{MODEL_DIR}/{batch_name}_ResNet50.keras",
    f"{MODEL_DIR}/{batch_name}_VGG16.keras",
    f"{MODEL_DIR}/{batch_name}_InceptionV3.keras",
    f"{MODEL_DIR}/{batch_name}_EfficientNetB0.keras",
    f"{MODEL_DIR}/{batch_name}_MobileNetV2.keras",
]

loaded_models = []
for path in model_paths:
    if os.path.exists(path):
        print(f"Loading {os.path.basename(path)}")
        try:
            # Pass the custom object when loading individual models
            m = load_model(path, custom_objects={'focal_loss_fixed': focal_loss()})
            m.trainable = False
            loaded_models.append(m)
        except Exception as e:
            print(f"Error loading model {os.path.basename(path)}: {e}")
    else:
        print(f"Model not found: {path}")

# --- Create Ensemble ---
if len(loaded_models) > 1:
    input_tensor = Input(shape=(224, 224, 3))
    outputs = [m(input_tensor) for m in loaded_models]
    avg_output = Average()(outputs)
    ensemble_model = Model(inputs=input_tensor, outputs=avg_output, name="EnsembleModel")

    # Compile
    ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Save the ensemble model
    ensemble_path = f"{MODEL_DIR}/{batch_name}_Ensemble.keras"
    ensemble_model.save(ensemble_path)
    print(f"\nEnsemble model saved at: {ensemble_path}")
else:
    print("No models found to ensemble.")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Define and register the custom focal loss function
@tf.keras.utils.register_keras_serializable()
def focal_loss_fixed(y_true, y_pred, gamma=2., alpha=0.25):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    bce_exp = tf.exp(-bce)
    return alpha * (1 - bce_exp) ** gamma * bce


ensemble_model = load_model(
    "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results/SplitBalanced_batch_3_Ensemble.keras",
    custom_objects={'focal_loss_fixed': focal_loss_fixed}
)

IMG_SIZE = (224, 224)
img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/defective/Physical (33).jpg"

img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = ensemble_model.predict(img_array)[0][0]
label = "Non-Defective" if prediction >= 0.5 else "Defective"

plt.imshow(img_array[0])
plt.title(f"{label} ({prediction:.2f})")
plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Define and register the custom focal loss function
@tf.keras.utils.register_keras_serializable()
def focal_loss_fixed(y_true, y_pred, gamma=2., alpha=0.25):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    bce_exp = tf.exp(-bce)
    return alpha * (1 - bce_exp) ** gamma * bce


ensemble_model = load_model(
    "/content/drive/MyDrive/solarPanelDefectDetection_Binary/balancedClasses/model_results/SplitBalanced_batch_3_Ensemble.keras",
    custom_objects={'focal_loss_fixed': focal_loss_fixed}
)

IMG_SIZE = (224, 224)
img_path = "/content/drive/MyDrive/solarPanelDefectDetection_Binary/binaryClassification/test/defective/Physical (64).jpg"

img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = ensemble_model.predict(img_array)[0][0]
label = "Non-Defective" if prediction >= 0.5 else "Defective"

plt.imshow(img_array[0])
plt.title(f"{label} ({prediction:.2f})")
plt.axis('off')
plt.show()